In [1]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.optimizers import RMSprop
from keras import initializers
from keras import regularizers
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.min_rows = 100
pd.options.display.max_columns = 100
# pd.options.display.min_columns = 100

Using TensorFlow backend.


In [2]:
batch_size = 271
original_dim = 327
latent_dim = 20
intermediate_dim = 100
epochs = 500
epsilon_std = 1
z_log_sigma_prior = np.log(0.7)

In [3]:
# encoder #
x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)

z_log_sigma_input = Input(batch_shape = (batch_size, 1))
z_log_sigma = Dense(units = 1,  activation = "linear",
                    kernel_initializer=initializers.Ones(),
                    use_bias = False)(z_log_sigma_input)

In [4]:
# sampling from latent space #
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim))
    return z_mean + K.exp(z_log_sigma) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

In [5]:
# decoder #
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

In [6]:
# end-to-end autoencoder
vae = Model([x, z_log_sigma_input], x_decoded_mean)

# encoder, from inputs to latent space
encoder_z_mean = Model([x, z_log_sigma_input], z_mean)
encoder_z_log_sigma = Model([x, z_log_sigma_input], z_log_sigma)

# generator, from latent space to reconstructed inputs
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

In [7]:
def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * binary_crossentropy(x, x_decoded_mean, label_smoothing=0.1)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss

In [8]:
def recon_metric(x, x_decoded_mean):
    xent_loss = original_dim * binary_crossentropy(x, x_decoded_mean, label_smoothing=0.1)
    return xent_loss

In [9]:
# vae.summary()

In [10]:
opt = optimizers.RMSprop(learning_rate=0.0005)
# opt = optimizers.Adam(learning_rate=0.001)
vae.compile(optimizer=opt, loss=vae_loss, metrics = [recon_metric])

In [11]:
movie = pd.read_csv('movie_analysis/movie.csv')
cast = movie.iloc[:,26:].to_numpy()
log_sigma_input = np.full((cast.shape[0], 1), z_log_sigma_prior)

In [12]:
import copy
test_manip = np.empty(shape=(327*2, 327))
idx = np.random.choice(a=list(range(2439)), size=327*2, replace=False)
for i in range(327):
    t1 = copy.deepcopy(cast[idx[2*i]])
    t1[i] = 1
    test_manip[(2*i)] = t1
    t0 = copy.deepcopy(cast[idx[2*i+1]])
    t0[i] = 0   
    test_manip[(2*i+1)] = t0
test_manip = test_manip[np.random.choice(a=list(range(327*2)), size=271*2, replace=False)]

In [13]:
from sklearn.model_selection import train_test_split
cast_train, test_obs = train_test_split(cast,train_size=1897,test_size = 542)
cast_test = np.concatenate((test_obs, test_manip), axis = 0)
log_sigma_input_train = np.full((cast_train.shape[0], 1), z_log_sigma_prior)
log_sigma_input_test = np.full((cast_test.shape[0], 1), z_log_sigma_prior)

In [14]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_loss', patience=3)

In [15]:
# fitting VAE #
history = vae.fit([cast_train,log_sigma_input_train], cast_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=([cast_test, log_sigma_input_test], cast_test),
        callbacks = [callback])

Train on 1897 samples, validate on 1084 samples
Epoch 1/500
1897/1897 [==============================] - 1s 447us/step - loss: 219.4717 - recon_metric: 219.4366 - val_loss: 210.6537 - val_recon_metric: 210.6160
Epoch 2/500
1897/1897 [==============================] - 0s 44us/step - loss: 205.2290 - recon_metric: 205.1896 - val_loss: 197.7479 - val_recon_metric: 197.7028
Epoch 3/500
1897/1897 [==============================] - 0s 43us/step - loss: 192.0981 - recon_metric: 192.0499 - val_loss: 183.5200 - val_recon_metric: 183.4619
Epoch 4/500
1897/1897 [==============================] - 0s 42us/step - loss: 177.0443 - recon_metric: 176.9813 - val_loss: 166.6585 - val_recon_metric: 166.5799
Epoch 5/500
1897/1897 [==============================] - 0s 42us/step - loss: 159.4468 - recon_metric: 159.3613 - val_loss: 147.3400 - val_recon_metric: 147.2316
Epoch 6/500
1897/1897 [==============================] - 0s 41us/step - loss: 139.9386 - recon_metric: 139.8223 - val_loss: 127.3171 - val_re

1897/1897 [==============================] - 0s 40us/step - loss: 72.3582 - recon_metric: 72.2652 - val_loss: 72.9954 - val_recon_metric: 72.9059
Epoch 104/500
1897/1897 [==============================] - 0s 43us/step - loss: 72.3539 - recon_metric: 72.2618 - val_loss: 72.9868 - val_recon_metric: 72.8988
Epoch 105/500
1897/1897 [==============================] - 0s 41us/step - loss: 72.3510 - recon_metric: 72.2596 - val_loss: 72.9783 - val_recon_metric: 72.8907
Epoch 106/500
1897/1897 [==============================] - 0s 40us/step - loss: 72.3427 - recon_metric: 72.2518 - val_loss: 72.9712 - val_recon_metric: 72.8844
Epoch 107/500
1897/1897 [==============================] - 0s 43us/step - loss: 72.3202 - recon_metric: 72.2300 - val_loss: 72.9640 - val_recon_metric: 72.8772
Epoch 108/500
1897/1897 [==============================] - 0s 41us/step - loss: 72.3193 - recon_metric: 72.2292 - val_loss: 72.9567 - val_recon_metric: 72.8703
Epoch 109/500
1897/1897 [=============================

1897/1897 [==============================] - 0s 39us/step - loss: 71.5129 - recon_metric: 71.2174 - val_loss: 72.3263 - val_recon_metric: 72.0614
Epoch 206/500
1897/1897 [==============================] - 0s 41us/step - loss: 71.5037 - recon_metric: 71.2059 - val_loss: 72.3214 - val_recon_metric: 72.0549
Epoch 207/500
1897/1897 [==============================] - 0s 40us/step - loss: 71.4891 - recon_metric: 71.1895 - val_loss: 72.3148 - val_recon_metric: 72.0463
Epoch 208/500
1897/1897 [==============================] - 0s 40us/step - loss: 71.4870 - recon_metric: 71.1849 - val_loss: 72.3099 - val_recon_metric: 72.0391
Epoch 209/500
1897/1897 [==============================] - 0s 39us/step - loss: 71.4807 - recon_metric: 71.1761 - val_loss: 72.3048 - val_recon_metric: 72.0322
Epoch 210/500
1897/1897 [==============================] - 0s 39us/step - loss: 71.4753 - recon_metric: 71.1684 - val_loss: 72.3010 - val_recon_metric: 72.0263
Epoch 211/500
1897/1897 [=============================

1897/1897 [==============================] - 0s 40us/step - loss: 70.9365 - recon_metric: 70.4498 - val_loss: 71.8971 - val_recon_metric: 71.4634
Epoch 308/500
1897/1897 [==============================] - 0s 40us/step - loss: 70.9284 - recon_metric: 70.4403 - val_loss: 71.8925 - val_recon_metric: 71.4577
Epoch 309/500
1897/1897 [==============================] - 0s 40us/step - loss: 70.9209 - recon_metric: 70.4314 - val_loss: 71.8959 - val_recon_metric: 71.4602
Epoch 310/500
1897/1897 [==============================] - 0s 40us/step - loss: 70.9175 - recon_metric: 70.4270 - val_loss: 71.8915 - val_recon_metric: 71.4547
Epoch 311/500
1897/1897 [==============================] - 0s 40us/step - loss: 70.9127 - recon_metric: 70.4208 - val_loss: 71.8850 - val_recon_metric: 71.4467
Epoch 312/500
1897/1897 [==============================] - 0s 40us/step - loss: 70.9151 - recon_metric: 70.4215 - val_loss: 71.8804 - val_recon_metric: 71.4401
Epoch 313/500
1897/1897 [=============================

In [16]:
# training loss #
print(min(history.history['loss']))
print(min(history.history['recon_metric']))
# validation loss #
print(min(history.history['val_loss']))
print(min(history.history['val_recon_metric']))

70.43514905657086
69.84023
71.58706474304199
71.05921173095703


In [17]:
# the fitted weight of input log_sigma #
encoder_z_log_sigma.layers[1].get_weights()[0][0,0]

# # retrieve weights from layer
# kernels, biases = model.layers[1].get_weights()

2.2496827

In [18]:
# the output log_sigma by structure#
z_log_sigma_prior*encoder_z_log_sigma.layers[1].get_weights()[0][0][0]

-0.8024054383728791

In [19]:
# the output log_sigma #
u_log_sigma = encoder_z_log_sigma.predict([cast, log_sigma_input], batch_size = batch_size)
print(u_log_sigma[0,0])
# the output sigma(u|t) #
u_t_sigma = np.exp(u_log_sigma[0,0])
u_t_sigma

-0.8024054


0.44824946

## Check relative magnitude of Var(U|t) to Var(U) using raw VAE estimates

In [20]:
# the output E(u|t) #
u_t_mean = encoder_z_mean.predict([cast, log_sigma_input], batch_size = batch_size)
u_t_mean

array([[ 0.2061964 , -0.27822572, -0.4040263 , ...,  0.89288056,
         1.0858158 , -0.10606455],
       [-1.0527308 ,  2.0027316 , -0.186849  , ...,  0.9433446 ,
         0.7240402 ,  0.10336753],
       [-1.1160333 ,  0.7293955 ,  0.09491093, ..., -0.5680689 ,
         0.48090318,  0.48628736],
       ...,
       [-0.8363727 , -0.00471742, -0.6216428 , ..., -0.8315696 ,
         0.31920627,  0.91461366],
       [-0.00572765, -0.56658196, -0.10645335, ...,  0.6082126 ,
         1.335349  , -0.2517928 ],
       [ 0.41068113, -0.553258  , -1.2883542 , ...,  0.584105  ,
         0.02087088,  0.13221157]], dtype=float32)

In [21]:
# marginal variance of U, way 1 #
mu_u = np.mean(u_t_mean, axis=0)
mu2_u = np.mean(u_t_mean**2+u_t_sigma**2, axis=0)  ##np.add() element-wise, broadcasting capability
var_u = mu2_u - mu_u**2
var_u.mean()

1.0792897

In [22]:
# marginal variance of U, way 2 #
var_u = np.var(u_t_mean) + u_t_sigma**2
var_u.mean()

1.0804223043907228

In [23]:
# Var(U) #
print(var_u.mean()) 
# Var(U|t) #
print(u_t_sigma**2) 
# Var(U|t)/Var(U) #
print(u_t_sigma**2 / var_u.mean())

1.0804223043907228
0.2009275777328554
0.18597133446459485


# Important Sampling Estimates (ISE)

In [24]:
from scipy.stats import norm
from scipy.stats import multivariate_normal

In [25]:
def f_t_u_ise(tr):
    nsim = 100
    u_t_mean = encoder_z_mean.predict([tr.reshape(1,original_dim), np.array([[z_log_sigma_prior]])])
    u_samples = np.random.multivariate_normal(mean = u_t_mean[0], \
                                          cov = u_t_sigma*np.identity(latent_dim), size = nsim)
    # f(t|u) #
    p_t_u = generator.predict(u_samples) ## in rows
    log_f_t_u = pd.DataFrame(p_t_u).apply(lambda x: sum(tr*np.log(x)+(1-tr)*np.log(1-x)), axis=1) ## in rowss
    # f(u) #
    log_f_u = multivariate_normal(mean=[0]*latent_dim, cov=np.identity(latent_dim)).logpdf(u_samples)
    # q(u|t) #
    log_q_u_t = multivariate_normal(mean = u_t_mean[0], cov = u_t_sigma*np.identity(latent_dim)).logpdf(u_samples)
    w = np.exp((log_f_t_u + log_f_u - log_q_u_t).to_numpy().reshape(nsim,1))
    weight = np.exp(np.log(w) - np.log(w.mean()))
    mu_u_t_ise = (u_samples*weight).mean(axis=0)
    mu_uu_t_ise = (np.apply_along_axis(lambda x: np.outer(x,x),1,u_samples)*weight.reshape(nsim,1,1)).mean(axis=0)
    cov_u_t_ise = mu_uu_t_ise - np.outer(mu_u_t_ise, mu_u_t_ise)
    result = np.array([u_t_mean[0], mu_u_t_ise, cov_u_t_ise, weight.reshape(nsim)], dtype=object)
    return result

In [26]:
ise_results = np.apply_along_axis(f_t_u_ise, 1, cast)

In [27]:
mu_u_t_output = np.empty((0, latent_dim))
mu_u_t_ise = np.empty((0, latent_dim))
cov_u_t_ise = np.empty((0, latent_dim, latent_dim))
for i in range(len(ise_results)):
    mu_u_t_output = np.append(mu_u_t_output, ise_results[i][0].reshape(1,latent_dim), axis = 0)
    mu_u_t_ise = np.append(mu_u_t_ise, ise_results[i][1].reshape(1,latent_dim), axis = 0)
    cov_u_t_ise = np.append(cov_u_t_ise, ise_results[i][2].reshape(1,latent_dim,latent_dim), axis = 0)
weight_mat = [ise_results[i][3] for i in range(len(ise_results))]

In [28]:
# output E(u|t) #
mu_u_t_output.mean(axis=0)

array([ 0.00061253, -0.03413393, -0.08745396, -0.01406428, -0.00711983,
       -0.06644257,  0.00708421, -0.00185817,  0.01604293,  0.04348041,
        0.03768379,  0.02021706,  0.01763638,  0.0252924 ,  0.03236644,
        0.01677274, -0.04113277, -0.01849788,  0.02317716,  0.01720955])

In [29]:
# ISE E(u|t) #
mu_u_t_ise.mean(axis=0)

array([-0.18633376, -0.25080738, -0.1213929 ,  0.006694  ,  0.24153515,
       -0.31169259,  0.01420833,  0.0157116 ,  0.22638571,  0.05581714,
        0.27022286,  0.30915915,  0.08307526,  0.28379361, -0.0660148 ,
        0.1381864 , -0.25882415, -0.0095447 ,  0.21199943,  0.12272114])

In [30]:
# output Var(u|t) #
print(u_t_sigma**2)

0.2009275777328554


In [31]:
# ISE Cov(u|t) #
cov_u_t_ise_ave = cov_u_t_ise.mean(axis=0)
cov_u_t_ise_ave

array([[ 3.95676847e-01, -1.05012235e-02, -2.42265256e-04,
         6.79349178e-03,  2.27642072e-02, -1.16612043e-03,
        -1.34566547e-03,  1.38951703e-02,  8.70330441e-03,
         3.24504487e-04,  1.36789857e-02,  1.73277648e-02,
        -1.07837835e-03,  5.18847640e-03, -2.43100198e-03,
         6.01186150e-03, -1.06426671e-02,  2.60358365e-03,
         8.22193104e-03,  2.81729941e-03],
       [-1.05012235e-02,  3.66128352e-01, -1.64990048e-02,
        -5.94037088e-04,  8.06409274e-03, -7.07807011e-03,
        -2.22803331e-03, -6.87251867e-03,  7.01788649e-03,
        -3.34650791e-03,  1.68357932e-02,  7.55763698e-03,
         5.17251800e-03,  1.21083591e-02, -8.94502994e-03,
         6.28942495e-03, -5.63732267e-03,  2.84119392e-03,
        -3.96417684e-05,  6.34969623e-05],
       [-2.42265256e-04, -1.64990048e-02,  4.02068342e-01,
        -1.72150771e-03,  3.08815287e-03,  1.82593980e-03,
        -6.30095763e-03,  1.46835001e-03, -5.64139850e-03,
         1.09906169e-02, -6.6

In [32]:
# Cov(u) = E(Cov(u|t)) + Cov(E(u|t))
cov_u = cov_u_t_ise.mean(axis=0) + np.cov(np.transpose(mu_u_t_ise))
print(cov_u)

[[ 1.41455424e+00  2.13708688e-02  4.43562014e-02  3.91031077e-02
   6.75806170e-02  6.87061870e-02  1.58968645e-02  6.99373502e-02
  -4.01806455e-02  8.56718506e-03  2.36863999e-02  1.58072040e-02
   4.73444122e-03 -3.80606623e-02  1.12844208e-02 -8.39720778e-05
   4.21877109e-02 -1.16208777e-02 -4.82258186e-02 -6.12344979e-02]
 [ 2.13708688e-02  1.21341866e+00 -2.37033295e-02  6.33593845e-02
  -6.71376297e-02  5.00013286e-04  4.41053886e-03 -1.90127167e-02
  -3.82259624e-02  2.06212456e-03 -6.27525401e-02 -5.53600724e-02
  -5.58016470e-02 -2.56390974e-03  3.92343943e-02 -4.29383197e-02
   6.25185779e-02  2.00716773e-03 -8.92775709e-02 -4.68978518e-02]
 [ 4.43562014e-02 -2.37033295e-02  1.39947488e+00 -3.53142516e-02
   6.18657837e-02  1.23348502e-01 -5.75773301e-02  1.35029747e-01
  -5.38384395e-02  1.05110757e-01 -5.48350221e-02 -7.19092055e-03
   1.25659908e-02 -1.72604634e-02 -4.88089060e-02  2.47990114e-02
  -2.45725201e-02  4.06460091e-02 -2.61239639e-04 -1.73040532e-02]
 [ 3.91

## Check relative magnitude of Var(U|t) to Var(U) using ISE estimates

In [33]:
# var(u|t)/var(u) for each dimension of u #
np.diag(cov_u_t_ise_ave)/np.diag(cov_u)

array([0.2797184 , 0.30173292, 0.28729944, 0.31731622, 0.27449068,
       0.2866884 , 0.29884616, 0.30883479, 0.28989962, 0.31038217,
       0.28049667, 0.28226784, 0.30205475, 0.29118187, 0.30372725,
       0.29590131, 0.27096457, 0.29398311, 0.29479157, 0.31185162])

In [34]:
# var(u|t)/var(u) averaging over dimension #
(np.diag(cov_u_t_ise_ave)/np.diag(cov_u)).mean()

0.29412146742656764

In [ ]:
import os
os.chdir("movie_analysis/vaedata+sensanaly")
os.getcwd()

In [39]:
pd.DataFrame(mu_u_t_ise).to_csv('mu_u_t_ise.csv', index = False)

In [40]:
pd.DataFrame(cov_u_t_ise_ave).to_csv('cov_u_t_ise.csv', index = False)

## Compute $E(U|Ti=1) - E(U|Ti=0)$ by raw VAE estimates

In [42]:
import copy
def cal_u_t_diff_org(i):
    t_cast1 = copy.deepcopy(cast)
    t_cast1[:,i] = 1
    u_t1 = encoder_z_mean.predict([t_cast1, log_sigma_input], batch_size = batch_size)
    t_cast0 = copy.deepcopy(cast)
    t_cast0[:,i] = 0
    u_t0 = encoder_z_mean.predict([t_cast0, log_sigma_input], batch_size = batch_size)
    return (u_t1 - u_t0).mean(axis = 0)

In [43]:
u_t_diff_allcast_org = [cal_u_t_diff_org(i) for i in range(cast.shape[1])]

In [44]:
pd.DataFrame(u_t_diff_allcast_org).to_csv('u_t_diff_allcast_org.csv', index = False)

## Compute $E(U|Ti=1) - E(U|Ti=0)$ by ISE estimates

In [71]:
def cal_mu_u_t_ise(tr):
    nsim = 1000
    u_t_mean = encoder_z_mean.predict([tr.reshape(1,original_dim), np.array([[z_log_sigma_prior]])])
    u_samples = np.random.multivariate_normal(mean = u_t_mean[0], \
                                          cov = u_t_sigma*np.identity(latent_dim), size = nsim)
    # f(t|u) #
    p_t_u = generator.predict(u_samples) ## in rows
    log_f_t_u = pd.DataFrame(p_t_u).apply(lambda x: sum(tr*np.log(x)+(1-tr)*np.log(1-x)), axis=1) ## in rows
    # f(u) #
    log_f_u = multivariate_normal(mean=[0]*latent_dim, cov=np.identity(latent_dim)).logpdf(u_samples)
    # q(u|t) #
    log_q_u_t = multivariate_normal(mean = u_t_mean[0], cov = u_t_sigma*np.identity(latent_dim)).logpdf(u_samples)
    w = np.exp((log_f_t_u + log_f_u - log_q_u_t).to_numpy().reshape(nsim,1))
    weight = np.exp(np.log(w) - np.log(w.mean()))
    mu_u_t_ise = (u_samples*weight).mean(axis=0)
    return mu_u_t_ise

In [72]:
import copy
def cal_u_t_diff(i):
    print(i)
    t_cast1 = copy.deepcopy(cast)
    t_cast1[:,i] = 1
    u_t1 = np.apply_along_axis(cal_mu_u_t_ise, 1, t_cast1)
    t_cast0 = copy.deepcopy(cast)
    t_cast0[:,i] = 0
    u_t0 = np.apply_along_axis(cal_mu_u_t_ise, 1, t_cast0)
    return (u_t1 - u_t0).mean(axis = 0)

In [73]:
sig_actors_index = (pd.read_csv('sig_actors_index.csv')-1)['x'].to_list()
np.array(sig_actors_index)

array([  0,   3,   9,  11,  14,  15,  27,  33,  40,  46,  55,  57,  68,
        69,  72,  74,  79,  83,  88,  99, 101, 105, 118, 122, 125, 137,
       148, 152, 154, 159, 171, 173, 174, 183, 185, 204, 226, 230, 244,
       249, 256, 268, 279, 284, 304, 316])

In [74]:
# actors' name with significant observed estimates #
(movie.columns.values[26:])[sig_actors_index]

array(['Zoe Saldana', 'Johnny Depp', 'Judi Dench', 'Michael Caine',
       'Morgan Freeman', 'Liam Neeson', 'Stan Lee', 'Mark Ruffalo',
       'Andy Serkis', 'Timothy Spall', 'Carla Gugino', 'Kate Bosworth',
       'Will Smith', 'Tommy Lee Jones', 'Ian McKellen', 'Hugo Weaving',
       'Kathy Bates', 'Leonardo DiCaprio', 'Judy Greer',
       'John Ratzenberger', 'Frank Welker', 'Jamie Foxx', 'Tom Hanks',
       'Arnold Schwarzenegger', 'Brad Pitt', 'Harrison Ford',
       'Jim Carrey', 'Channing Tatum', 'Rose Byrne', 'Reese Witherspoon',
       'Tom Cruise', 'Jeremy Piven', 'Dennis Hopper', 'Angelina Jolie',
       'Eddie Murphy', 'Elias Koteas', 'Robin Williams', 'John Travolta',
       'Bruce Willis', 'Octavia Spencer', 'Tim Blake Nelson',
       'Susan Sarandon', 'Viggo Mortensen', 'Denzel Washington',
       'Adam Sandler', 'Kevin Hart'], dtype=object)

In [75]:
u_t_diff_ise_sigactors = [cal_u_t_diff(i) for i in sig_actors_index]

0
3
9
11
14
15
27
33
40
46
55
57
68
69
72
74
79
83
88
99
101
105
118
122
125
137
148
152
154
159
171
173
174
183
185
204
226
230
244
249
256
268
279
284
304
316


In [76]:
pd.DataFrame(u_t_diff_ise_sigactors).to_csv('u_t_diff_ise_sigactors.csv', index = False)